<a href="https://colab.research.google.com/github/KiarashS/WeakMedSAM/blob/main/WeakMedSAM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)

2.6.0+cu124


In [2]:
import torchvision
print(torchvision.__version__)

0.21.0+cu124


In [3]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !conda install pytorch torchvision cudatoolkit=11.8 -c pytorch

In [4]:
!python -m pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 13.4 MB/s eta 0:00:00


In [5]:
!pip install -q kaggle

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kiarashs","key":"dd43ac962ffe7d1775c5b38aface305d"}'}

In [7]:
!mkdir ~/.kaggle

In [8]:
!cp kaggle.json ~/.kaggle/

In [9]:
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
# !kaggle datasets list

In [11]:
!kaggle datasets download 'aryashah2k/brain-tumor-segmentation-brats-2019'

Dataset URL: https://www.kaggle.com/datasets/aryashah2k/brain-tumor-segmentation-brats-2019
License(s): CC0-1.0
100% 2.59G/2.60G [00:26<00:00, 236MB/s]
100% 2.60G/2.60G [00:26<00:00, 104MB/s]


In [12]:
!git clone https://github.com/KiarashS/WeakMedSAM.git

Cloning into 'WeakMedSAM'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 65 (delta 17), reused 41 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 81.47 KiB | 3.39 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [13]:
!mkdir WeakMedSAM/brats-input

In [14]:
%%capture
!unzip brain-tumor-segmentation-brats-2019.zip -d WeakMedSAM/brats-input

In [15]:
!mv WeakMedSAM/brats-input/MICCAI_BraTS_2019_Data_Training/* WeakMedSAM/brats-input

In [16]:
!rm -rf WeakMedSAM/brats-output/

In [17]:
!python WeakMedSAM/brats/preprocess.py --input-path WeakMedSAM/brats-input/ --output-path WeakMedSAM/brats-output/

100%|█████████████████████████████████████████████████████████████| 335/335 [01:18<00:00,  4.29it/s]


In [18]:
!pip install scikit-learn

In [ ]:
%%capture
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
#!pip install --upgrade torch torchvision

In [ ]:
# %pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

In [20]:
!python WeakMedSAM/cluster.py --batch_size 256 --data_path WeakMedSAM/brats-output --data_module brats --parent_classes 1 --child_classes 8 --save_path WeakMedSAM/cluster-file --gpus 1

Namespace(data_path='WeakMedSAM/brats-output', save_path='WeakMedSAM/cluster-file', data_module='brats', batch_size=256, parent_classes=1, child_classes=8, gpus='1')
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|███████████████████████████████████████| 203/203 [1:23:23<00:00, 24.65s/it]


In [21]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

--2025-06-12 10:09:13--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.239.50.18, 18.239.50.104, 18.239.50.9, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.239.50.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M  40.2MB/s    in 5.9s    

2025-06-12 10:09:19 (60.8 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]



In [ ]:
# import tensorflow as tf
# tf.test.gpu_device_name()

In [25]:
!python WeakMedSAM/train.py --seed 42 --sam_ckpt sam_vit_b_01ec64.pth --lr '1e-4' --batch_size 24 --max_epochs 10 --val_iters 3000 --index mymodelindex --data_path WeakMedSAM/brats-output/ --data_module brats --parent_classes 1 --child_classes 8 --child_weight 0.5 --cluster_file WeakMedSAM/cluster-file/brats-8.bin --logdir WeakMedSAM/logdir --gpus 1

2025-06-12 10:15:42.210215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749723342.528360   26051 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749723342.613096   26051 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-12 10:15:43.276066: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Namespace(data_path='WeakMedSAM/brats-output/', data_module='brats', vit_name=None, sam_ckpt='sam_vit_b_01ec64.pth', 

In [24]:
!python WeakMedSAM/lab_gen.py --batch-size 24 --data-path WeakMedSAM/brats_output --save-path WeakMedSAM/pseudo_label --data-module brats --parent-classes 1 --child-classes 8 --samus-ckpt WeakMedSAM/logdir/mymodelindex/mymodelindex.pth --sam-ckpt sam_vit_b_01ec64.pth --t 4 --beta 4 --threshold 0.5 --gpus 1

Namespace(data_path='WeakMedSAM/brats_output', save_path='WeakMedSAM/pseudo_label', data_module='brats', vit_name=None, sam_ckpt='sam_vit_b_01ec64.pth', batch_size=24, samus_ckpt='WeakMedSAM/logdir/mymodelindex/mymodelindex.pth', parent_classes=1, child_classes=8, t=4, beta=4, threshold=0.5, gpus='1')
Traceback (most recent call last):
  File "/content/WeakMedSAM/lab_gen.py", line 97, in <module>
    dataset = data_module.get_all_dataset(args.data_path, 0, "")
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/WeakMedSAM/brats/dataset.py", line 189, in get_all_dataset
    *get_files(
     ^^^^^^^^^^
  File "/content/WeakMedSAM/brats/dataset.py", line 177, in get_files
    num_files = len(os.listdir(os.path.join(data_path, sample_name))) // 2
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'WeakMedSAM/brats_output/BraTS19_2013_0_1'


In [ ]:
!python WeakMedSAM/train_unet.py --seed 42 --lr '1e-4' --batch_size 128 --max_epochs 10 --val_iters 500 --index mymodelindex --data_path WeakMedSAM/brats_output --lab_path WeakMedSAM/pseudo_label --data_module brats --num_classes 2 --logdir WeakMedSAM/logdir --gpus 1

In [ ]:
!python WeakMedSAM/eval.py --data_path WeakMedSAM/brats_output --data_module 'brats' --batch_size 128 --num_classes 2 --ckpt WeakMedSAM/logdir/mymodelindex/mymodelindex.pth --gpus 1